In [1]:
import pandas as pd
import numpy as np
import re
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer #Pretrained model for text embedding
from torchviz import make_dot
from tqdm import tqdm

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def clean_text(text):
    """
    Clean text and convert to lowercase
    """
    return re.sub(r'[^\w\s]', '',text).lower()

def Preprocessing(df):
    """
    Wrapper Function to Preprocess the data (replacing NaNs,
    encoding non-numeric columns to numeric values)
    """
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    try:
        df['overall_sentiment']= df['overall_sentiment'].map({'neutral': 0, 'positive': 1,'negative':2})      
    except:
        pass
    
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
def Preprocessingtest(df):
    """
    Wrapper Function to Preprocess the data (replacing NaNs,
    encoding non-numeric columns to numeric values)
    """
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
        
    
def get_max_length_words(df,col):
    """
    Gets max number of words in DF column, for padding purposes
    """
    try:
        return df[col].str.split("\\s+").str.len().max()
    except:
        RaiseError("Invalid Column passed")
                
def padding(tensor,length):
    """
    Performs padding
    """
    tensor=torch.tensor(tensor) if not torch.is_tensor(tensor) else tensor
    return torch.nn.functional.pad(tensor,(0, length-tensor.size()[0]))

In [4]:
class MemeDataset(Dataset):
    """
    Custom Dataloader for our dataset.
    All credit goes to: https://github.com/aladdinpersson/Machine-Learning-Collection/blame/master/ML/Pytorch/Basics/custom_dataset_txt/loader_customtext.py
    """
    def __init__(self, dataframe, root_dir, pad_length,model,transform = None, freq_threshold = 5):
        self.df = dataframe.copy()
        self.img = self.df['image_name']
        self.pad_len=pad_length
        self.captions = self.df['text_corrected']
        self.root_dir = root_dir
        self.transform = transform
        self.model=model

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        #convert image to 3 channel RGB, as some images in our dataset are >3 channels or <3 channels
        image = Image.open(os.path.join(self.root_dir, self.df.iloc[index, 0])).convert("RGB")
        y_label = torch.tensor(int(self.df.iloc[index, 2]))
        caption = self.captions[index]
        if self.transform:
            image = self.transform(image)   
        padded_text_tensor = padding(self.model.encode(caption,convert_to_tensor=True),self.pad_len)
        return (image, padded_text_tensor, y_label)

In [5]:
class NN_img(nn.Module):
    """
    Network for images
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(100*100*3, 1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,60),
            nn.Tanh(),
            nn.Linear(60,3)
        )
            
    def forward(self, x):
        return self.network(x)
    
class NN_text(nn.Module):
    """
    Network for text
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
        nn.Flatten(),
        nn.Linear(200, 160),
        nn.ReLU(),
        nn.Linear(160,128),
        nn.ReLU(),
        nn.Linear(128,72),
        nn.Tanh(),
        nn.Linear(72,64),
        nn.Tanh(),
        nn.Linear(64,3)
        )

    def forward(self, x):
        return self.network(x)    
    
def __train__(dataloader, model, loss_fn, optimizer,flag):
    """
    Trains network on trainset
    """
    size = len(dataloader.dataset)
    for batch, (img, txt, y) in enumerate(dataloader):
        img=img.to(device)
        txt=txt.to(device)
        y=y.to(device)
        if (flag==0):
            pred = model(img)
        else:
            pred = model(txt)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch*len(img)
        print(f"loss: {loss:>2f} [{current:>5d}/{size:>2d}]")


def __valid__(dataloader, model, loss_fn,acc_list,flag):
    """
    Performs evaluation on testset
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            if (flag==0):
                pred = model(img)
            else:
                pred = model(txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Average loss: {test_loss:>2f} \n")
    acc_list.append(100*correct)
def __test__(dataloader, model,label,flag):
    """
    Performs evaluation on testset
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    res=[]
    number=[]

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            if (flag==0):
                pred = model(img)
            else:
                pred = model(txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            print(y.cpu().numpy())
            res.append(pred_txt)
            number.append(y.cpu().numpy())
    return res,number

In [6]:
# 导入os模块
import os
from pandas import DataFrame
path = "C:\\Users\\86187\\AI\\finalproject\\data"
data={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number=[]
sentiment=[]
# os.listdir()方法获取文件夹名字，返回数组
with open("C:\\Users\\86187\\AI\\finalproject\\train.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number.append(s[0])
            sentiment.append(s[1])
        f.close()
del(number[0])
del(sentiment[0])
file_name_list = os.listdir(path)
img=[]
txt=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img.append(file_name_list[i])
    else:
        txt.append(file_name_list[i])
for i in range(len(img)):
    z=str(img[i]).split('.')
    if z[0] in number:
        data['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt[i]),'r',encoding="ANSI") as f:
            for line in f:
                data['text_corrected'].append(line)
            f.close()
            data['overall_sentiment'].append(sentiment[number.index(z[0])])
df1=pd.DataFrame(data)
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,How I feel today #legday #jelly #aching #gym \n,positive
1,10.jpg,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.jpg,This is my Valentine's from 1 of my nephews. I...,positive
3,1001.jpg,Zoe's first love #Rattled @JohnnyHarper15 \n,positive
4,1002.jpg,Chaotic Love - giclee print ?65 at #art #love ...,positive
...,...,...,...
3995,995.jpg,OMG. Well done #Eskom! 'Man dies during #LoadS...,positive
3996,996.jpg,Feelin' the love in here! #ValentinesDay #cari...,positive
3997,997.jpg,#blue #eyes can't be #beaten \n,positive
3998,998.jpg,LA CHUCHA LOUUU TE CHUPO LOS OJOS..! \n,positive


In [7]:
data1={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number1=[]
sentiment1=[]
with open("C:\\Users\\86187\\AI\\finalproject\\test_without_label.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number1.append(s[0])
            sentiment1.append(s[1])
        f.close()
del(number1[0])
del(sentiment1[0])
file_name_list = os.listdir(path)
img1=[]
txt1=[]
label=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img1.append(file_name_list[i])
    else:
        txt1.append(file_name_list[i])
for i in range(len(img1)):
    z=str(img1[i]).split('.')    
    if z[0] in number1:
        data1['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt1[i]),'r',encoding="ANSI") as f:
            for line in f:
                data1['text_corrected'].append(line)
            f.close()
            data1['overall_sentiment'].append(z[0])
            label.append(z[0])
df2=pd.DataFrame(data1)
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [8]:
Preprocessing(df1) #Cleaning NaNs, text, encoding sentiment to numeric values
Preprocessingtest(df2)
df1
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [9]:
alpha_len=170 #arbitrary value to add for padding on top of max length sentence
alpha_len1=173
pad_len=get_max_length_words(df1,'text_corrected')+alpha_len
pad_len1=get_max_length_words(df2,'text_corrected')+alpha_len1
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,how i feel today legday jelly aching gym \n,1
1,10.jpg,arrivatw absolute disgrace two carriages from ...,2
2,100.jpg,this is my valentines from 1 of my nephews i a...,1
3,1001.jpg,zoes first love rattled johnnyharper15 \n,1
4,1002.jpg,chaotic love giclee print 65 at art love chao...,1
...,...,...,...
3995,995.jpg,omg well done eskom man dies during loadsheddi...,1
3996,996.jpg,feelin the love in here valentinesday caring \n,1
3997,997.jpg,blue eyes cant be beaten \n,1
3998,998.jpg,la chucha louuu te chupo los ojos \n,1


In [10]:
vector_model=SentenceTransformer('all-MiniLM-L6-v2').to(device) #Pretrained model to generate text embeddings
#All credit to: https://www.sbert.net/docs/pretrained_models.html
dataset = MemeDataset(dataframe = df1, root_dir = './data', pad_length=pad_len,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))
test_set = MemeDataset(dataframe = df2, root_dir = './data', pad_length=pad_len1,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))

In [11]:
#Loading train and test sets
train_set, valid_set = torch.utils.data.random_split(dataset, [3000, 1000])
dataloader_train = DataLoader(dataset = train_set, batch_size = 32, shuffle=True)
dataloader_valid = DataLoader(dataset = valid_set, batch_size = 32, shuffle=True)
dataloader_test = DataLoader(dataset = test_set, batch_size = 32, shuffle=True)

In [12]:
#Setting up Neural Networks
net_img = NN_img().to(device) #image network
net_text = NN_text().to(device) #text network 

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net_img.parameters(), lr=0.001)

epochs = 10
acc=list()
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n_______________________________")
    __train__(dataloader_train, net_img, loss_fn, optimizer,0)
    __valid__(dataloader_valid, net_img, loss_fn,acc,0)
    if(t>=2 and t%2==0):
            print(f"Accuracy of t-3 iter: {acc[t-2]}\nAccuracy of current iter: {acc[t]}")
print(f"Highest accuracy was: {max(acc)}")
result=[]
numberlabel=[]
result,numberlabel=__test__(dataloader_test, net_img,label,0)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
_______________________________
loss: 1.131576 [    0/3000]
loss: 1.023131 [   32/3000]
loss: 1.203324 [   64/3000]
loss: 1.006655 [   96/3000]
loss: 0.803313 [  128/3000]
loss: 0.902923 [  160/3000]
loss: 0.907522 [  192/3000]
loss: 1.000338 [  224/3000]
loss: 0.827025 [  256/3000]
loss: 0.913456 [  288/3000]
loss: 0.781049 [  320/3000]
loss: 0.949774 [  352/3000]
loss: 1.204023 [  384/3000]
loss: 1.336222 [  416/3000]
loss: 0.749891 [  448/3000]
loss: 0.850630 [  480/3000]
loss: 0.912452 [  512/3000]
loss: 1.006774 [  544/3000]
loss: 0.800567 [  576/3000]
loss: 0.867251 [  608/3000]
loss: 1.110059 [  640/3000]
loss: 0.855732 [  672/3000]
loss: 0.862575 [  704/3000]
loss: 0.968373 [  736/3000]
loss: 1.029436 [  768/3000]
loss: 0.990674 [  800/3000]
loss: 0.911480 [  832/3000]
loss: 0.994441 [  864/3000]
loss: 0.972918 [  896/3000]
loss: 1.087729 [  928/3000]
loss: 0.905853 [  960/3000]
loss: 0.988495 [  992/3000]
loss: 0.841885 [ 1024/3000]
loss: 0.814020 [ 1056/3000]
loss: 0.

 10%|████████▎                                                                          | 1/10 [01:13<11:03, 73.69s/it]

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1']
Test Error: 
 Accuracy: 61.00%, Average loss: 0.901016 

Epoch 2
_______________________________
loss: 1.081703 [    0/3000]
loss: 0.755133 [   32/3000]
loss: 0.917682 [   64/3000]
loss: 0.940375 [   96/3000]
loss: 0.996593 [  128/3000]
loss: 0.819550 [  160/3000]
loss: 0.857440 [  192/3000]
loss: 0.849615 [  224/3000]
loss: 0.848894 [  256/3000]
loss: 0.990070 [  288/3000]
loss: 0.844291 [  320/3000]
loss: 0.887845 [  352/3000]
loss: 0.920802 [  384/3000]
loss: 0.780179 [  416/3000]
loss: 0.793618 [  448/3000]
loss: 0.961646 [  480/3000]
loss: 0.965717 [  512/3000]
loss: 0.882308 [  544/3000]
loss: 0.864118 [  576/3000]
loss: 0.965497 [  608/3000]
loss: 0.636340 [  640/3000]
loss: 0.988526 [  672/3000]
loss: 1.023269 [  704/3000]
loss: 0.798848 [  736/3000]
loss: 0.932650 [  768/3000]
l

 20%|████████████████▌                                                                  | 2/10 [02:29<10:00, 75.01s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [11:20<01:15, 75.99s/it]

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1']
Test Error: 
 Accuracy: 61.00%, Average loss: 0.893972 

Accuracy of t-3 iter: 61.0
Accuracy of current iter: 61.0
Epoch 10
_______________________________
loss: 0.874299 [    0/3000]
loss: 1.070988 [   32/3000]
loss: 0.907937 [   64/3000]
loss: 0.776685 [   96/3000]
loss: 0.698646 [  128/3000]
loss: 0.683649 [  160/3000]
loss: 1.091739 [  192/3000]
loss: 0.855017 [  224/3000]
loss: 1.060746 [  256/3000]
loss: 0.835398 [  288/3000]
loss: 1.137096 [  320/3000]
loss: 0.930124 [  352/3000]
loss: 0.898013 [  384/3000]
loss: 0.897646 [  416/3000]
loss: 1.070486 [  448/3000]
loss: 1.052768 [  480/3000]
loss: 0.866778 [  512/3000]
loss: 0.793871 [  544/3000]
loss: 0.866756 [  576/3000]
loss: 0.835466 [  608/3000]
loss: 0.865422 [  640/3000]
loss: 1.085430 [  672/3000]
loss: 1.065806 [  704/3000

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [12:36<00:00, 75.64s/it]

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1']
Test Error: 
 Accuracy: 61.00%, Average loss: 0.890907 

Highest accuracy was: 61.0


['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
[1227 2367 2863 3998 5000 4796 3842 4287 3082 1682 1737   55 3346  460
 3701  383 3102 3397 4461 2555 3714 1233 3003 1381  534   60 4912 5108
 2635  780 2607 4885]
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
[ 349 1301 1875 4064 4112 1099 1393 3731 2746   13 1476  494 1435  224
 3039  459 4417  803 1895 3651 1228 3580 4945 2677 1898 3833 3292 5039
 4807 3909 4621 3174]
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
[4242 3271  716 3891  176 1285  259 1868 5097 2676  916 1431  848 2537
 3279 4777 1918 4795 1205 3110 1324 1048 1037 3374 1352 1635 3495 1306
 2114 1642 1485 3854]
['1', '1', '1', '1', '1',

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net_text.parameters(), lr=0.001)
acc=list()
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n_______________________________")
    __train__(dataloader_train, net_text, loss_fn, optimizer,1)
    __valid__(dataloader_valid, net_text, loss_fn,acc,1)
    if(t>=2 and t%2==0):
            print(f"Accuracy of t-3 iter: {acc[t-2]}\nAccuracy of current iter: {acc[t]}")
print(f"Highest accuracy was: {max(acc)}")
result=[]
numberlabel=[]
result,numberlabel=__test__(dataloader_test, net_text,label,1)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
_______________________________
loss: 1.131056 [    0/3000]
loss: 1.119124 [   32/3000]
loss: 1.116910 [   64/3000]
loss: 1.102834 [   96/3000]
loss: 1.096138 [  128/3000]
loss: 1.092207 [  160/3000]
loss: 1.077840 [  192/3000]
loss: 1.060951 [  224/3000]
loss: 1.055938 [  256/3000]
loss: 1.035444 [  288/3000]
loss: 1.011462 [  320/3000]
loss: 0.981555 [  352/3000]
loss: 1.043718 [  384/3000]
loss: 1.020195 [  416/3000]
loss: 0.903608 [  448/3000]
loss: 0.957304 [  480/3000]
loss: 1.022632 [  512/3000]
loss: 0.851896 [  544/3000]
loss: 0.920881 [  576/3000]
loss: 0.918706 [  608/3000]
loss: 0.981480 [  640/3000]
loss: 0.953518 [  672/3000]
loss: 0.966914 [  704/3000]
loss: 1.072205 [  736/3000]
loss: 1.000231 [  768/3000]
loss: 0.744952 [  800/3000]
loss: 1.069496 [  832/3000]
loss: 0.796551 [  864/3000]
loss: 0.768042 [  896/3000]
loss: 0.877278 [  928/3000]
loss: 1.108473 [  960/3000]
loss: 1.032836 [  992/3000]
loss: 0.937573 [ 1024/3000]
loss: 0.913598 [ 1056/3000]
loss: 0.

 10%|████████▎                                                                          | 1/10 [01:00<09:04, 60.54s/it]

['1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '2', '2', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '2', '2']
Test Error: 
 Accuracy: 65.40%, Average loss: 0.768475 

Epoch 2
_______________________________
loss: 0.853612 [    0/3000]
loss: 0.756212 [   32/3000]
loss: 0.801834 [   64/3000]
loss: 1.022310 [   96/3000]
loss: 0.957043 [  128/3000]
loss: 0.635752 [  160/3000]
loss: 0.761786 [  192/3000]
loss: 0.699678 [  224/3000]
loss: 0.764726 [  256/3000]
loss: 0.816524 [  288/3000]
loss: 0.654264 [  320/3000]
loss: 0.662125 [  352/3000]
loss: 0.660804 [  384/3000]
loss: 0.629865 [  416/3000]
loss: 0.797802 [  448/3000]
loss: 0.814327 [  480/3000]
loss: 0.872413 [  512/3000]
loss: 0.672269 [  544/3000]
loss: 0.675557 [  576/3000]
loss: 0.665849 [  608/3000]
loss: 0.694809 [  640/3000]
loss: 0.725324 [  672/3000]
loss: 0.795748 [  704/3000]
loss: 0.805649 [  736/3000]
loss: 0.718966 [  768/3000]
l

 20%|████████████████▌                                                                  | 2/10 [01:58<07:53, 59.21s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [09:26<01:06, 66.18s/it]

['2', '1', '0', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '2', '0', '2', '2', '1', '1', '1', '1', '2', '1', '2', '1']
['1', '1', '2', '1', '1', '1', '1', '2']
Test Error: 
 Accuracy: 65.10%, Average loss: 1.467691 

Accuracy of t-3 iter: 66.9
Accuracy of current iter: 65.10000000000001
Epoch 10
_______________________________
loss: 0.058967 [    0/3000]
loss: 0.038402 [   32/3000]
loss: 0.071369 [   64/3000]
loss: 0.049682 [   96/3000]
loss: 0.288722 [  128/3000]
loss: 0.286523 [  160/3000]
loss: 0.221447 [  192/3000]
loss: 0.027018 [  224/3000]
loss: 0.033819 [  256/3000]
loss: 0.033967 [  288/3000]
loss: 0.187046 [  320/3000]
loss: 0.025776 [  352/3000]
loss: 0.325745 [  384/3000]
loss: 0.137434 [  416/3000]
loss: 0.205627 [  448/3000]
loss: 0.063906 [  480/3000]
loss: 0.155630 [  512/3000]
loss: 0.028411 [  544/3000]
loss: 0.049718 [  576/3000]
loss: 0.037117 [  608/3000]
loss: 0.047162 [  640/3000]
loss: 0.036922 [  672/3000]
loss: 0.10782

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [10:37<00:00, 63.71s/it]

['1', '0', '0', '2', '2', '1', '2', '2', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '2', '1']
['1', '2', '1', '1', '1', '2', '1', '2']
Test Error: 
 Accuracy: 64.50%, Average loss: 1.580832 

Highest accuracy was: 68.2


['1', '1', '1', '0', '1', '1', '0', '2', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '2', '1', '1', '2', '1', '1', '0', '2', '1', '1', '1', '2', '1', '1']
[3505 3572 3745 3451 1025 1497 2676 2764 1301 1039 1620 1552 3300 1866
 3424 3052   87 4541 1574 5040 3397 2597  803 3637 3914  848  964 1918
 4871 2746 3248 4201]
['0', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '2', '1', '0', '2', '1', '1', '1', '1', '2', '1', '2', '1', '1', '2', '2', '1', '1']
[2085 1228 5092 3346 1587 2463  863  609 4272 4843 1933 5009 3580 2480
 3005 1269 3174 4790 4587 1252 1476 1993  812 3854 2991 3298 1949  145
 4603 1908 3046 2717]
['2', '1', '1', '1', '2', '1', '1', '2', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '2', '1', '1', '1', '0', '1', '1', '1', '0', '1']
[2325 3969 2115 3348 1512 4094 3206 4471  286 3554 1173 2348 1999 3651
 4771 4555   31 2709 3494 3243 3292  916 3462 2127 3660 5027 2546 3008
 1290 4920  881 1697]
['1', '0', '1', '2', '0',